In [1]:
from agents import Agent, Runner, trace, gen_trace_id, function_tool, WebSearchTool
from agents.model_settings import ModelSettings
import sendgrid
from pydantic import BaseModel, Field
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
import os
from dotenv import load_dotenv
import gradio as gr 
from typing import Dict

In [2]:
load_dotenv(override=True)

True

# Agents

In [3]:
default_model = "gpt-4o-mini"

## Searcher Agent

In [4]:
# agent instruction's parameters
instructions_params = {
    "paragraphs": 3,
    "words": 300,
    "tone": "efficient"
} 

# instruction set
instructions = f"""
You are a research assistant.

Task: Search the web for the given term and write a summary.  
Length: {instructions_params["paragraphs"]} paragraphs, about {instructions_params["words"]} words.  
Tone: {instructions_params["tone"]}.  

Guidelines:  
- Focus only on key findings and main points.  
- Exclude fluff, filler, or personal commentary.  
- Output only the summary text.  
"""

# agent properties
searcher_agent = Agent(
    name = "Searcher Agent",
    instructions = instructions,
    tools = [WebSearchTool(search_context_size="low")], 
    model = default_model,
    model_settings = ModelSettings(tool_choice = "required")
)

## Search Optimiser Agent

In [5]:
# Agent instruction's parameters
# Due to the costs of using WebSearchTool, i will use only 3 searchs. 
instructions_params = {
    "terms": 3,
} 

# instruction set
instructions = f"""
You are an optimization research assistant.

Task: Given a query, generate {instructions_params["terms"]} web search terms.  
Guidelines:  
- Focus on terms that best answer the original query.  
- Output only the search terms (no explanations or extra text).  
"""

# Using the Pydantic Functions as an easier way to package json scripts. 

# Give the reasoning for picking the term, based on the query
class WebSearchTerm(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")

# This class will store the list of the x questions/concepts.
class WebSearchPlan(BaseModel):
    searches: list[WebSearchTerm] = Field(description="A list of web searches to perform to best answer the query.")

# agent properties
search_optimiser_agent = Agent(
    name = "Search Optimiser Agent",
    instructions = instructions,
    model = default_model,
    output_type = WebSearchPlan
)

## Writer Agent

In [6]:
# agent instruction's parameters
instructions_params = {
    "page_range": [6, 8],
    "words": 1000,
    "tone": "cohesive"
} 

# instruction set
instructions = f"""
You are a senior researcher.

Task: Write a {instructions_params["tone"]} research report based on a query and initial research.  

Steps:  
1. Create a clear outline showing the structure and flow.  
2. Write the full report in markdown format.  

Requirements:  
- Length: {instructions_params["page_range"][0]}–{instructions_params["page_range"][1]} pages (~{instructions_params["words"]} words).  
- Style: Detailed, thorough, and structured.
- Output only the outline and the final markdown report (no extra commentary).  
"""

# Will generate a json based strucutre based on the generate report
# This will be very helpful for the emailer agent. 
class ReportData(BaseModel):
    short_summary: str = Field(description=
                               "A short 2-3 sentence summary of the findings.")
    markdown_report: str = Field(description="The markdown final report.")
    follow_up_questions: str = Field(description="Suggested topics to research further.")

# agent properties
writer_agent = Agent(
    name = "Writer Agent",
    instructions = instructions,
    model = default_model,
    output_type = ReportData
)

## Emailer

In [7]:
# agent instruction's parameters
instructions_params = {
    "format": "professional and nice",
} 

# instruction set
instructions = f"""
You are an assistant that sends {instructions_params["format"]}-formatted HTML emails.

Task: Convert the provided detailed report into a single HTML email and send it using the send_email tool.  

Requirements:  
- Subject line must be clear and relevant.  
- Body must be clean, well-structured HTML.  
- Call the send_email tool once with the subject and HTML body.  
- Do not output anything else.  
"""

# A function which is used for transporting the findings in the form of a report (report being completed by the writer agent).
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("kthonnithodi@gmail.com")
    to_email = To("kthonnithodi@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content)
    mail_json = mail.get()
    response = sg.client.mail.send.post(request_body=mail_json)
    return {"status": "success"}

# agent properties
emailer_agent = Agent(
    name = "Emailer Agent",
    instructions = instructions,
    tools = [send_email], 
    model = "gpt-4o-mini"
)

# Functions

In [8]:
async def plan_searches(query: str):
    """Calling search_optimiser to find an instance of prompt for the query"""
    print("Planning searches...")
    result = await Runner.run(search_optimiser_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def search(item: WebSearchTerm):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(searcher_agent, input)
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Calling search_optimiser to discover x amount of prompts based on the query provided."""
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent.writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(emailer_agent, report.markdown_report)
    print("Email sent")
    return report

# Main Program

In [21]:
query = "NVDA Earnings, Income statements, balance sheets from start of 2025 to end of 2025."

with trace("Starting Search Plan"):
    results = await plan_searches(query=query)

print(results)

Planning searches...
Will perform 3 searches
searches=[WebSearchTerm(reason='To find detailed earnings reports for NVIDIA throughout 2025.', query='NVIDIA NVDA earnings report 2025'), WebSearchTerm(reason='To access comprehensive income statements for NVIDIA in 2025.', query='NVIDIA NVDA income statement 2025'), WebSearchTerm(reason='To locate balance sheets for NVIDIA for the year 2025.', query='NVIDIA NVDA balance sheet 2025')]
